# Deepfake Detection Challenge

1. Download dataset by using chrome inspector. Click the 'all.zip' and cancel the download. Then in chrome inspector use copy as cURL to copy to SageMaker
2. Figure out opencv to convert the mp4's to images
3. Balance out the dataset since it has more fakes than reals.

In [14]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [15]:
from fastai.vision import *
import json
import pandas as pd
import cv2
import os

video_path = '/home/ec2-user/.fastai/data/sample/train_sample_videos'
image_path = os.makedirs(os.path.join(video_path, 'images'), exist_ok=True)

labels = pd.read_json(open(os.path.join(video_path,'metadata.json')))
labels = labels.transpose()
labels['filename'] = labels.axes[0].tolist()
cols = ['filename','label','split','original']
labels = labels[cols]
labels.index = list(range(labels.shape[0]))
labels.to_csv(os.path.join(video_path, 'labels.csv'))
video_list = labels['filename']

In [16]:
labels['split'].value_counts()

train    400
Name: split, dtype: int64

In [17]:
labels.describe()

,filename,label,split,original
count,400,400,400,323
unique,400,2,1,209
top,cqhngvpgyi.mp4,FAKE,train,atvmxvwyns.mp4
freq,1,323,400,6


In [19]:
def videoConverter(video):
    vidcap = cv2.VideoCapture(os.path.join(video_path,video))
    def getFrame(sec):
        vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
        hasFrames,image = vidcap.read()
        if hasFrames:
            image_path = '/home/ec2-user/.fastai/data/sample/train_sample_videos/images/'
            cv2.imwrite(os.path.join(image_path, video + '_' + str(count) + ".jpg"), image)
        return hasFrames
    sec = 0
    frameRate = 0.5 #//it will capture image in each 0.5 second
    count=1
    success = getFrame(sec)
    while success:
        count = count + 1
        sec = sec + frameRate
        sec = round(sec, 2)
        success = getFrame(sec)

        
for video in video_list:
    videoConverter(video)

In [20]:
import shutil

In [42]:
images_dir = '/home/ec2-user/.fastai/data/sample/train_sample_videos/images'
fakes_dir = '/home/ec2-user/.fastai/data/sample/train_sample_videos/images/fakes'
reals_dir = '/home/ec2-user/.fastai/data/sample/train_sample_videos/images/reals'

os.mkdir(fakes_dir)
os.mkdir(reals_dir)

# copy the fakes to the fakes directory
fnames = [os.listdir(images_dir)][0]
for fname in fnames:
    src = os.path.join(images_dir, fname)
    dst = os.path.join(fakes_dir, fname)
    shutil.copyfile(src, dst)
    
    
# copy the reals to the reals directory
#for fname in fnames:
#    src = os.path.join(images_dir, fname)
#    dst = os.path.join(reals_dir, fname)
#    shutil.copyfile(src, dst)
    

IsADirectoryError: [Errno 21] Is a directory: '/home/ec2-user/.fastai/data/sample/train_sample_videos/images/fakes'

In [34]:
images_dir = '/home/ec2-user/.fastai/data/sample/train_sample_videos/images'
fnames = [os.listdir(images_dir)][0]

In [41]:
test_video = fnames[2].split('_', 1)[0]
test_video

'ehccixxzoe.mp4'